In [ ]:
!pip install torch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0 --index-url https://download.pytorch.org/whl/cu121
!pip uninstall mamba-ssm causal-conv1d
!pip install causal-conv1d && pip install mamba-ssm

Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 799.1/799.1 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 62.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/ex_comparison_forecasting

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import os
import gc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.stats import norm
import math
import sklearn.preprocessing
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import *
import pickle
from scipy import interpolate
import random
import time
from lightgbm import LGBMRegressor
# import optuna
from sklearn.metrics import mean_absolute_error
from sklearn.multioutput import MultiOutputRegressor
from scipy.stats import linregress

import argparse
import torch
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
import torch.nn.functional as F
import pandas as pd
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import mean_absolute_percentage_error as MAPE
from sklearn.metrics import root_mean_squared_error as RMSE
from sklearn.metrics import r2_score as R2
from torchinfo import summary
from mamba_ssm import Mamba

In [ ]:
with open(f'list_EQ_id.p', 'rb') as f:
    list_EQid = pickle.load(f)


In [ ]:
def getPositionEncoding(seq_len, d, n=10000):
    P = np.zeros((seq_len, d))
    for k in range(seq_len):
        for i in np.arange(int(d/2)):
            denominator = np.power(n, 2*i/d)
            P[k, 2*i] = np.sin(k/denominator)
            P[k, 2*i+1] = np.cos(k/denominator)
    return P


In [ ]:
def prepare_data_batch(x, N1, N2, list_output_time, j = 2):
  N0 = np.shape(x)[1]
  xxx = np.stack([x[:,i:N0 - N1+i+1:j, :] for i in range(N1)], axis=2)
  y = xxx[:,:,-N2:,-len(list_output_time):]

  return np.array(xxx), np.array(y)


In [ ]:
def data_provider2(data_time, data_static, data_y, batch_size=128):
    data_set = h_Dataset(data_time, data_static, data_y)
    data_loader = DataLoader(data_set, batch_size=batch_size, shuffle=True, drop_last=True)
    return data_set, data_loader


In [ ]:
class h_Dataset(Dataset):
    def __init__(self, data_time, data_static, data_y):
        self.data_time = data_time
        self.data_static = data_static
        self.data_y = data_y

    def __getitem__(self, index):
        seq_time = self.data_time[index]
        seq_static = self.data_static[index]
        seq_y = self.data_y[index]
        return seq_time, seq_static, seq_y

    def __len__(self):
        return len(self.data_y)



In [ ]:
class TokenEmbedding(nn.Module):
    def __init__(self, c_in, d_model):
        super(TokenEmbedding, self).__init__()
        self.tokenConv = nn.Conv1d(in_channels=c_in, out_channels=d_model, kernel_size=3, padding=1, padding_mode='circular', bias=False)

    def forward(self, x):
        x = self.tokenConv(x.permute(0, 2, 1)).transpose(1, 2)
        return x

class DataEmbedding(nn.Module):
    def __init__(self, c_in, d_model, dropout=0.1):
        super(DataEmbedding, self).__init__()
        self.value_embedding = TokenEmbedding(c_in=c_in, d_model=d_model)
        self.position_embedding = PositionalEmbedding(d_model=d_model)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x):
          x = self.value_embedding(x) + self.position_embedding(x)
          return self.dropout(x)

class PositionalEmbedding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEmbedding, self).__init__()

        pe = torch.zeros(max_len, d_model).float()
        pe.require_grad = False
        position = torch.arange(0, max_len).float().unsqueeze(1)
        div_term = (torch.arange(0, d_model, 2).float() * -(np.log(10000.0) / d_model)).exp()

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return self.pe[:, :x.size(1)]


In [ ]:
class Data_aggregation(nn.Module):
    def __init__(self, L_all, N_static, dropout=0.1):
        super(Data_aggregation, self).__init__()
        self.projection = nn.Sequential(nn.Linear(N_static, L_all), nn.ReLU(), nn.Linear(L_all, L_all))
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x_time, x_static):
        x = self.projection(x_static)
        x = self.dropout(x)
        x = x.permute(0, 2, 1)
        x = torch.cat([x_time, x], dim=2)

        return x



In [ ]:
def gen_data_AI_batch_2_model(df_new, list_idx, L_all, L_out, L_ground, N_input, N_output, N_static, sliding_step, list_time_series, list_output_time, list_static, real_scaler, static_scaler):
  L_in = L_all - L_out
  signal = [np.stack(list(df_new[k][list_idx])) for k in list_time_series ]
  signal = np.stack(signal).transpose(1,2,0)
  signal = np.pad(signal,((0,0),(L_all,0),(0,0)),constant_values=0.0)
  signal = signal.reshape(-1, N_input + N_output)
  scaled_signal = real_scaler.transform(signal)
  scaled_signal = scaled_signal.reshape(-1, L_ground+L_all, N_input + N_output)
  X_times_series, Y_label = prepare_data_batch(scaled_signal, L_all, L_out, list_output_time, j = sliding_step)
  X_times_series[:,:,L_in:,N_input:N_input+N_output] = 0.0

  static = np.stack(list(df_new[k][list_idx] for k in list_static)).T
  scaled_static = static_scaler.transform(static)
  X_static = scaled_static.reshape(-1,1,1,N_static)
  X_static = np.tile(X_static, [1, X_times_series.shape[1], 1, 1])
  X_times_series = X_times_series.reshape(-1, L_all, N_input+N_output)
  X_static = X_static.reshape(-1, 1, N_static)
  Y_label =  Y_label.reshape(-1, L_out, N_output)

  return X_times_series, X_static, Y_label


In [ ]:
def FullAttention(queries, keys, values):
      B, L, H, E = queries.shape
      _, S, _, D = values.shape
      scale = 1. / math.sqrt(E)
      scores = torch.einsum("blhe,bshe->bhls", queries, keys)
      A = torch.softmax(scale * scores, dim=-1)
      V = torch.einsum("bhls,bshd->blhd", A, values)
      return V.contiguous()


class AttentionLayer(nn.Module):
    def __init__(self, attention, d_model, dropout, n_heads=4):
        super(AttentionLayer, self).__init__()
        d_keys = d_model // n_heads
        d_values = d_model // n_heads
        self.attention = attention
        self.query_projection = nn.Linear(d_model, d_keys * n_heads)
        self.key_projection = nn.Linear(d_model, d_keys * n_heads)
        self.value_projection = nn.Linear(d_model, d_values * n_heads)
        self.out_projection = nn.Linear(d_values * n_heads, d_model)
        self.n_heads = n_heads
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, L, _ = x.shape
        _, S, _ = x.shape
        H = self.n_heads
        queries = self.query_projection(x).view(B, L, H, -1)
        keys = self.key_projection(x).view(B, S, H, -1)
        values = self.value_projection(x).view(B, S, H, -1)
        out = self.attention(queries,keys,values)
        out = out.view(B, L, -1)
        out = self.out_projection(out)
        x = self.dropout(F.relu(x + out))
        return x

class CNN(nn.Module):
    def __init__(self, d_model):
        super(CNN, self).__init__()
        self.conv = nn.Conv1d(in_channels=d_model, out_channels=d_model, kernel_size=3, padding=1)
        self.relu = nn.ReLU()

    def forward(self, x):
        # Encoder
        x = x.permute(0, 2, 1)  # Shape: [N_sample, N_timestep, d_model]
        x = self.conv(x)  # Shape: [N_sample, d_model, N_timestep]
        x = self.relu(x)
        x = x.permute(0, 2, 1)  # Shape: [N_sample, d_model, N_timestep]
        return x

class LSTM(nn.Module):
    def __init__(self, d_model):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(input_size=d_model, hidden_size=d_model)

    def forward(self, x):
        # Encoder
        x, _ = self.lstm(x)  # Shape: [N_sample, N_timestep / 2, 64]
        return x


class CNNLSTM(nn.Module):
    def __init__(self, d_model):
        super(CNNLSTM, self).__init__()
        self.conv = nn.Conv1d(in_channels=d_model, out_channels=d_model, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.lstm = nn.LSTM(input_size=d_model, hidden_size=d_model)

    def forward(self, x):
        # Encoder
        x = x.permute(0, 2, 1)  # Shape: [N_sample, N_timestep, d_model]
        x = self.conv(x)  # Shape: [N_sample, d_model, N_timestep]
        x = self.relu(x)
        x = x.permute(0, 2, 1)  # Shape: [N_sample, d_model, N_timestep]
        x, _ = self.lstm(x)  # Shape: [N_sample, N_timestep / 2, 64]
        return x

class Mamba_h_2(nn.Module):
    def __init__(self, configs):
        super(Mamba_h_2, self).__init__()
        self.mamba = Mamba(
            d_model = configs.d_model,
            d_state = 16,
            d_conv = 4,
            expand = 2,
        )


    def forward(self, x):
        x = self.mamba(x)
        return x

class ResBlock(nn.Module):
    def __init__(self, configs, L_all, N_all):
        super(ResBlock, self).__init__()


        self.temporal = nn.Sequential(
            nn.Linear(L_all, configs.d_model),
            nn.ReLU(),
            nn.Linear(configs.d_model, L_all),
            nn.Dropout(configs.dropout)
        )

        self.channel = nn.Sequential(
            nn.Linear(N_all, configs.d_model),
            nn.ReLU(),
            nn.Linear(configs.d_model, N_all),
            nn.Dropout(configs.dropout)
        )

    def forward(self, x):
        # x: [B, L, D]
        x = x + self.temporal(x.transpose(1, 2)).transpose(1, 2)
        x = x + self.channel(x)
        return x


class TSMixer_dis(nn.Module):
    def __init__(self, configs, L_all, L_out, N_input, N_output, mtype='Mamba'):
        super(TSMixer_dis, self).__init__()
        self.layer = configs.e_layers
        self.model = nn.ModuleList([])

        if mtype == 'MLP':
          self.model = nn.ModuleList([])
        elif mtype == 'TSMixer':
          self.model = nn.ModuleList([ResBlock(configs, L_all, N_input+N_output+1) for _ in range(configs.e_layers)])
        elif mtype == 'TSMixer-embed':
          self.model = nn.ModuleList([DataEmbedding(N_input+N_output+1, configs.d_model, configs.dropout)]
                                    +[ResBlock(configs, L_all, configs.d_model) for _ in range(configs.e_layers)]
                                    +[nn.Sequential(nn.Linear(configs.d_model, N_input+N_output+1), nn.ReLU())])
        elif mtype == 'Transformer':
          self.model = nn.ModuleList([DataEmbedding(N_input+N_output+1, configs.d_model, configs.dropout)]
                                    +[AttentionLayer(FullAttention, configs.d_model, configs.dropout) for _ in range(configs.e_layers)]
                                    +[nn.Sequential(nn.Linear(configs.d_model, N_input+N_output+1), nn.ReLU())])
        elif mtype == 'CNNLSTM':
          self.model = nn.ModuleList([DataEmbedding(N_input+N_output+1, configs.d_model, configs.dropout)]
                                    +[CNNLSTM(configs.d_model) for _ in range(configs.e_layers)]
                                    +[nn.Sequential(nn.Linear(configs.d_model, N_input+N_output+1), nn.ReLU())])
        elif mtype == 'CNN':
          self.model = nn.ModuleList([DataEmbedding(N_input+N_output+1, configs.d_model, configs.dropout)]
                                    +[CNN(configs.d_model) for _ in range(configs.e_layers)]
                                    +[nn.Sequential(nn.Linear(configs.d_model, N_input+N_output+1), nn.ReLU())])
        elif mtype == 'LSTM':
          self.model = nn.ModuleList([DataEmbedding(N_input+N_output+1, configs.d_model, configs.dropout)]
                                    +[LSTM(configs.d_model) for _ in range(configs.e_layers)]
                                    +[nn.Sequential(nn.Linear(configs.d_model, N_input+N_output+1), nn.ReLU())])
        elif mtype == 'Mamba':
          self.model = nn.ModuleList([DataEmbedding(N_input+N_output+1, configs.d_model, configs.dropout)]
                                    +[Mamba_h_2(configs) for _ in range(configs.e_layers)]
                                    +[nn.Sequential(nn.Linear(configs.d_model, N_input+N_output+1), nn.ReLU())])


        self.projection = nn.Sequential(nn.Linear(L_all, L_out), nn.ReLU())
        self.out_layer = nn.ModuleList([nn.Linear(N_input+N_output+1, N_output, bias=False) for i in range(7)])

    def forward(self, x_enc):
        x = x_enc
        for i in range(len(self.model)):
            x = self.model[i](x)

        x = self.projection(x.permute(0, 2, 1)).permute(0, 2, 1)
        x_out = torch.stack([self.out_layer[i](x) for i in range(7)], dim=-1)

        return x_out

class model_parameters:
  def __init__(self):
    self.d_model = 128
    self.e_layers = 2
    self.dropout = 0.1

# [0.02, 0.1, 0.25, 0.5, 0.75, 0.9, 0.98],
class MAE_2():
    def loss(y_pred, target):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        L = torch.zeros(1).to(device)
        for i, q in enumerate([0.02, 0.1, 0.25, 0.5, 0.75, 0.9, 0.98]):
          errors = y_pred[:,:,:,i] - target
          L += torch.mean(torch.max((q - 1) * errors, q * errors))

        return L


In [ ]:
def one_function_optim(ex_name, mtype='Mamba', N_epoch = 20, Pencode=True, verbose = 0):

    d_model, e_layers, L_in, L_out, dropout = 128, 2, 50, 750, 0.1
    L_all = L_in + L_out
    sliding_step = L_out

    with open(f'EQall.p','rb') as f:
      EQall = pickle.load(f)

    with open(f'new_p_data_allEQ_{ex_name}.p', 'rb') as f:
      df_new = pickle.load(f)

    for i in range(len(df_new)):
      L = len(df_new['disp_ground'][i])
      if L >3000:
        df_new['disp_ground'][i] = df_new['disp_ground'][i][:3000]
      else:
        df_new['disp_ground'][i] = list(df_new['disp_ground'][i]) + [0]*(3000-L)

    list_output_time = []
    list_static = []

    for k in list(df_new):
      if (str(k)[:4] == 'disp') & (str(k)[-1] != 'd'):
        list_output_time.append(k)

      if str(k)[:4] != 'disp':
        list_static.append(k)

    df_new['EQid'] = list_EQid
    list_len = np.array([len(df_new['disp_2'][ip]) for ip in range(len(df_new))])
    list_idx = np.where(list_len!=3000)[0]
    df_new = df_new.drop(index=list_idx)
    df_new = df_new.reset_index()


    L_signal_input = len(df_new['disp_ground'][0])
    L_ground = L_signal_input

    Ndim_encoding = 2
    P_encoding = np.zeros((L_signal_input, Ndim_encoding))
    if Pencode==True:
      P_encoding = getPositionEncoding(seq_len=L_signal_input, d=Ndim_encoding, n=10000)

    for d in range(Ndim_encoding):
      df_new[f'encode_{d}'] = [P_encoding[:,d]]*len(df_new)

    list_input_time = ['disp_ground']+[f'encode_{d}' for d in range(Ndim_encoding)]
    list_time_series = list_input_time + list_output_time
    L_in = L_all-L_out
    N_input= len(list_input_time)
    N_output= len(list_output_time)
    N_static = len(list_static)
    N_time_series = len(list_input_time)+len(list_output_time)+1

    static_scaler = sklearn.preprocessing.StandardScaler().fit(df_new[list_static].values)
    target = np.array([df_new[i][0] for i in list_output_time]).T
    target_scaler = sklearn.preprocessing.StandardScaler().fit(target)
    real = np.array([df_new[i][0] for i in list_time_series]).T
    real_scaler = sklearn.preprocessing.StandardScaler().fit(real)


    train_time, train_static, train_label = gen_data_AI_batch_2_model(df_new, idx_train, L_all, L_out, L_ground, N_input, N_output, N_static, sliding_step, list_time_series, list_output_time, list_static, real_scaler, static_scaler)
    val_time, val_static, val_label = gen_data_AI_batch_2_model(df_new, idx_valid, L_all, L_out, L_ground, N_input, N_output, N_static, sliding_step, list_time_series, list_output_time, list_static, real_scaler, static_scaler)
    _, train_loader = data_provider2(train_time, train_static, train_label, batch_size=64)
    _, valid_loader = data_provider2(val_time, val_static, val_label, batch_size=64)


    args = model_parameters()
    args.d_model = d_model
    args.e_layers = e_layers
    args.dropout = dropout

    model_tor = TSMixer_dis(args, L_all, L_out, N_input, N_output, mtype)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    aggregation = Data_aggregation(L_all, N_static, dropout)
    aggregation = aggregation.float().to(device)
    model_tor = model_tor.float().to(device)
    model_optim = torch.optim.Adam(list(model_tor.parameters())+list(aggregation.parameters()), lr=0.001)
    criterion = MAE_2

    time1 = time.time()

    for epoch in range(N_epoch):
        train_loss = []
        for batch_time, batch_static, batch_y in train_loader:
            model_optim.zero_grad()
            batch_time, batch_static, batch_y = batch_time.float().to(device), batch_static.float().to(device), batch_y.float().to(device)
            batch_x = aggregation(batch_time, batch_static)
            outputs = model_tor(batch_x)
            loss = criterion.loss(outputs, batch_y)
            loss.backward()
            train_loss.append(loss.item())
            model_optim.step()

        if verbose == 1:
          if epoch % 2 ==0:
            valid_loss = []
            for batch_time, batch_static, batch_y in valid_loader:
              batch_time, batch_static, batch_y = batch_time.float().to(device), batch_static.float().to(device), batch_y.float().to(device)
              batch_x = aggregation(batch_time, batch_static)
              outputs = model_tor(batch_x)
              loss = criterion.loss(outputs, batch_y)
              valid_loss.append(loss.item())

            print(f"epoch: {epoch} | loss: {np.mean(train_loss)}, valid loss: {np.mean(valid_loss)}")


    time2 = time.time()

    list_idx = idx_valid
    x_time, x_static, _ = gen_data_AI_batch_2_model(df_new, list_idx, L_all, L_out, L_ground, N_input, N_output, N_static, sliding_step, list_time_series, list_output_time, list_static, real_scaler, static_scaler)
    x_time = x_time.reshape(len(list_idx),-1,L_all, N_input+N_output)
    x_static = x_static.reshape(len(list_idx),-1, 1, N_static)

    y_predict_all = np.zeros((len(x_time), L_ground+L_all, N_output))

    for j in range(int(L_ground/L_out)+1):
      x_time[:,j,:L_all, N_input:N_input+N_output] = y_predict_all[:,L_out*j:L_all+L_out*j,:]
      xt = torch.from_numpy(x_time[:,j,:,:]).float().to('cuda')
      xs = torch.from_numpy(x_static[:,j,:,:]).float().to('cuda')
      batch_x = aggregation(xt, xs)
      y_pred_all = model_tor(batch_x).to('cuda').cpu().detach().numpy()
      y_predict_all[:,L_in+L_out*j:L_all+L_out*j,:] = y_pred_all[:,:,:,3]

    y_predict_all = target_scaler.inverse_transform(y_predict_all.reshape(-1, N_output))
    y_predict_all = y_predict_all.reshape(len(x_time), -1, N_output)


    dict_MAE = {k:[] for k in list_output_time}
    dict_MAPE = {k:[] for k in list_output_time}
    dict_RMSE = {k:[] for k in list_output_time}
    dict_R2 = {k:[] for k in list_output_time}

    for i_s in range(len(list_output_time)):
      for i in range(len(list_idx)):
        calMAE = MAE(df_new[list_output_time[i_s]][list_idx[i]], y_predict_all[i, L_all:L_all+L_ground,i_s])
        dict_MAE[list_output_time[i_s]].append(calMAE)
        calR2 = R2(df_new[list_output_time[i_s]][list_idx[i]], y_predict_all[i, L_all:L_all+L_ground,i_s])
        dict_R2[list_output_time[i_s]].append(calR2)

        calMAPE = MAPE(df_new[list_output_time[i_s]][list_idx[i]], y_predict_all[i, L_all:L_all+L_ground,i_s])
        dict_MAPE[list_output_time[i_s]].append(calMAPE)
        calRMSE = RMSE(df_new[list_output_time[i_s]][list_idx[i]], y_predict_all[i, L_all:L_all+L_ground,i_s])
        dict_RMSE[list_output_time[i_s]].append(calRMSE)



    mean_MAE = [np.mean(dict_MAE[k]) for k in dict_MAE]
    mean_MAPE = [np.mean(dict_MAPE[k]) for k in dict_MAPE]
    mean_RMSE = [np.mean(dict_RMSE[k]) for k in dict_RMSE]
    mean_R2 = [np.mean(dict_R2[k]) for k in dict_R2]

    time3 = time.time()

    total_params = sum(p.numel() for p in model_tor.parameters() if p.requires_grad)

    print(summary(model_tor, batch_x.size()))

    return mean_R2[-1], mean_MAE[-1], mean_MAPE[-1], mean_RMSE[-1], time1, time2, time3, total_params


In [ ]:
%%time
list_type = ['MLP', 'TSMixer', 'Transformer', 'CNNLSTM', 'CNN', 'LSTM', 'Mamba']
list_ex = ['ex1', 'ex2', 'ex3', 'ex4', 'ex5', '3Dex1', '3Dex2', '3Dex3', '3Dex4', '3Dex5']
for ex_name in list_ex:
  if len(ex_name)>3:
    Nex = 230
  else:
    Nex=250

  idx_train = np.random.choice(Nex, size=200, replace=False)
  idx_valid = [i for i in range(Nex) if i not in idx_train]

  for mtype in list_type:
    if 'noencode' in mtype:
      Pencode = False
    else:
      Pencode = True

    _, _, _, _, time1, time2, time3, total_params = one_function_optim(ex_name, mtype=mtype, N_epoch=1, Pencode = Pencode, verbose=0)
    # print(ex_name, mtype, (time2-time1)*10, time3-time2)
    print(ex_name, mtype, total_params)


In [ ]:
!pip install torchinfo

In [ ]:
from torchinfo import summary

In [ ]:
input_size=(64, 800, 7)
summary(model_tor, input_size)


Layer (type:depth-idx)                   Output Shape              Param #
TSMixer_dis                              [64, 750, 3, 7]           --
├─ModuleList: 1-1                        --                        --
│    └─DataEmbedding: 2-1                [64, 800, 128]            --
│    │    └─TokenEmbedding: 3-1          [64, 800, 128]            2,688
│    │    └─PositionalEmbedding: 3-2     [1, 800, 128]             --
│    │    └─Dropout: 3-3                 [64, 800, 128]            --
│    └─Mamba_h_2: 2-2                    [64, 800, 128]            --
│    │    └─Mamba: 3-4                   [64, 800, 128]            116,480
│    └─Mamba_h_2: 2-3                    [64, 800, 128]            --
│    │    └─Mamba: 3-5                   [64, 800, 128]            116,480
│    └─Sequential: 2-4                   [64, 800, 7]              --
│    │    └─Linear: 3-6                  [64, 800, 7]              903
│    │    └─ReLU: 3-7                    [64, 800, 7]              --
├

In [ ]:
%%time
list_type = ['MLP', 'TSMixer', 'Transformer', 'CNNLSTM', 'CNN', 'LSTM', 'Mamba']
list_ex = ['3Dex1', '3Dex2', '3Dex3', '3Dex4', '3Dex5']
idx_train = np.random.choice(230, size=200, replace=False)
idx_valid = [i for i in range(230) if i not in idx_train]

for ex_name in list_ex:
  for mtype in list_type:
    value_R2, value_MAE, value_MAPE, value_RMSE = one_function_optim(ex_name, mtype=mtype, N_epoch=200, Pencode = True, verbose=0)
    print(mtype, value_R2, value_MAE, value_MAPE, value_RMSE)


3Dex1
MLP -41.30167022929695 0.004930706913994626 54.86519013967485 0.007142784977770953
3Dex1
TSMixer -1.7081752377036175 0.002693610120664471 15.351447922294646 0.0038243283483156473
3Dex1
Transformer -1.9379068890026638 0.00367475644593396 227.83931493289438 0.005117187521910883
3Dex1
CNNLSTM -0.6536314208606203 0.0035777336646286104 15.974262502877327 0.0050171001055147
3Dex1
CNN -1.5152092340796561 0.004471504540059696 9.306769006012159 0.006507840409558719
3Dex1
LSTM -4.1013917679648 0.003958994948053806 20.181044928752318 0.005722930824058763
3Dex1
Mamba -64.1251116112397 0.006714151556961109 59.45187587026573 0.009292163314314582
3Dex3
MLP -0.39479537229373274 0.009075627643508277 62.9665461958797 0.013821717426227035
3Dex3
TSMixer 0.1321487538680185 0.0040206717331179595 68.36192657101164 0.00576866338784523
3Dex3
Transformer -0.25957298949196184 0.006737781973443727 87.09622548679822 0.009963915672502762
3Dex3
CNNLSTM -0.004354711770745668 0.01253231749863887 19.1208667049551